In [2]:
import pandas as pd

##### Reading data

In [3]:
#filename = 'D:\\Backup\\ISB CBA\\Capstone\\summary data\\transaction_summary.csv'

#data=pd.read_csv(filename)
#d={'quantity':'count','sales_quantity':'sum','purchase_quantity':'sum','transfer_quantity':'sum'}
#chunksize = 10 ** 5
#rows=0
#df_north = pd.DataFrame()
#for chunk in pd.read_csv(filename, chunksize=chunksize):
# 
#    n=chunk[chunk['state']=='ST03']
#    df_north=pd.concat([df_north,n])
#    rows+=chunk.shape[0]
    #print (rows)

In [ ]:
filename = 'D:\\Backup\\ISB CBA\\Capstone\\summary data\\summary_area_code_02.csv'
df_north=pd.read_csv(filename)

In [4]:
df_north.shape

(868026, 50)

In [5]:
df_north.columns

Index(['location_code', 'item_no', 'week', 'closing_date', 'state', 'region',
       'brand', 'quantity', 'purchase_quantity', 'transfer_quantity',
       'sales_quantity', 'purchase_cost_amount', 'purchase_mrp',
       'purchase_date', 'stock_prevailing_mrp', 'store_in',
       'product_group_code', 'transfer_cost_amount', 'sales_department',
       'days_to_sell', 'num_of_customers', 'total_price', 'line_discount',
       'crm_line_discount', 'discount', 'tax', 'cost', 'billing',
       'contribution', 'trade_incentive', 'trade_incentive_value',
       'total_contribution', 'store_type', 'store_location', 'city_type',
       'available_quantity', 'week_no', 'year', 'case_size', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type'],
      dtype='object')

In [27]:
#extracting specific columns

col=['location_code', 'item_no', 'state', 'region',
       'brand', 'stock_prevailing_mrp', 'store_type', 'store_location', 'city_type',
       'available_quantity', 'week_no', 'year', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type','billing','sales_quantity']

df_north_final=df_north.loc[:,col]
df_north_final.fillna(0, inplace=True)

df_north_final['item_no']=df_north_final['item_no'].astype(str)
#df_north_final['case_size']=df_north_final['case_size'].astype(str)
df_north_final['case_shape']=df_north_final['case_shape'].astype(str)

#### Defining a function for doing pareto analysis on features

The function aggregates all levels of a categorical features that account for ~10% by Sales billings into a new level called "others". Features with less than 10 levels are not considered for pareto analysis.

In [28]:
def pareto(cols):
    lst=[]
    for col in cols:
                
        series=df_north_final.fillna(0).groupby([col]).agg({'billing':'sum'}).sort_values('billing',ascending=False)
        mask=series.cumsum()/series.sum()>0.9 
        #nos=mask.value_counts()[1]
        mask=mask.iloc[:,0]
        levels=len(df_north_final[col].unique())
        if levels>10:
            df_north_final[col] = np.where(df_north_final[col].isin(series[mask].index),'Other',df_north_final[col])         
        new_levels=len(df_north_final[col].unique())

        freq=df_north_final[col].value_counts()/df_north_final[col].value_counts().sum()*100
        freq=freq.round(2)

        sale_qty=df_north_final.groupby([col]).agg({'sales_quantity':'sum'}).sort_values('sales_quantity',ascending=False)
        sale_qty=sale_qty/sale_qty.sum()*100
        sale_qty=sale_qty.round(2)
        try:
            Other_Sales_Qty=sale_qty['sales_quantity']['Other']
        except:
            Other_Sales_Qty=0
        
        mrp=df_north_final.groupby([col]).agg({'stock_prevailing_mrp':'sum'}).sort_values('stock_prevailing_mrp',ascending=False)
        mrp=mrp/mrp.sum()*100
        mrp=mrp.round(2)
        try:
            Other_MRP=mrp['stock_prevailing_mrp']['Other']
        except:
            Other_MRP=0
        
        #comparison=mrp.merge(sale_qty, left_index=True, right_index=True)
        lst.append([col.upper(),levels, new_levels,Other_MRP,Other_Sales_Qty])
        #print ("%s-Originally %d levels,combined %d levels into 'Other'.New Levels %d.By MRP,Other is %2.1f and by sale qty others is %2.1f"%(col.upper(),levels, levels-new_levels, new_levels,mrp['stock_prevailing_mrp']['Other'],sale_qty['sales_quantity']['Other']))
    
    cols=['Feature', 'Orig Levels', 'New Levels', 'Other%(MRP)', 'Other%(Sales Qty)']
    df = pd.DataFrame(lst, columns=cols)
    df=df.set_index("Feature")
    
    return df

In [29]:
import numpy as np
#cols=['brand','case_size', 'case_size_range', 'gender', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'watch_type']
cols=['case_size_range', 'gender','movement', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'case_shape', 'watch_type']
pareto(cols)

,Orig Levels,New Levels,Other%(MRP),Other%(Sales Qty)
Feature,,,,
CASE_SIZE_RANGE,13,6,21.40,20.31
GENDER,3,3,0.00,0.00
MOVEMENT,10,10,0.00,0.00
MATERIAL,80,9,17.04,20.33
DIAL_COLOR,63,11,13.79,9.82
STRAP_TYPE,79,11,12.40,13.83
STRAP_COLOR,54,7,15.41,21.50
PRECIOUS_STONE,9,9,0.00,0.00
GLASS,8,8,0.00,0.00


### Modelling

In [65]:
#sorting by week_no and year
df_north_final=df_north_final.sort_values(['week_no', 'year'], ascending=[True, False])

In [32]:
#creatign dummy variables
cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type', 'week_no', 'year']
df_north_dummies=pd.get_dummies(data=df_north_final, columns=cols)

In [33]:
df_north_dummies.shape

(868026, 272)

In [34]:
#!pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [54]:
df_north_dummies.head()

,location_code,item_no,stock_prevailing_mrp,available_quantity,billing,sales_quantity,brand_0,brand_B003,brand_B007,brand_B008,...,week_no_W47,week_no_W48,week_no_W49,week_no_W50,week_no_W51,week_no_W52,week_no_W53,year_FY1718,year_FY1819,year_FY1920
334,S02,5164126,7995.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
540,S02,5175912,46750.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
906,S02,5185430,6900.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1012,S02,5188753,15800.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1031,S02,5190965,168200.0,1,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
y=df_north_dummies.loc[:, df_north_dummies.columns == 'sales_quantity']
X=df_north_dummies.drop(columns =['sales_quantity','location_code','item_no'])

In [38]:
#checking for duplicate column names
duplicate_columns = df_north_dummies.columns[df_north_dummies.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [39]:
#performing train and test split on data

split=0.2
test=int(len(X)*split)
train=len(X)-test
X_train=X.head(train)
y_train=y.head(train)
X_test=X.tail(test)
y_test=y.tail(test)

#### Modelling using xgboost

In [47]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.652854


In [48]:
from sklearn.metrics import r2_score
r2_score(y_test,preds)

-0.00690198124545871

##### Tuning hyperparameters

In [ ]:
params = {'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'subsample': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'colsample_bylevel':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'min_child_weight ':[1,3,5,7] ,
          'reg_lambda':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'n_estimator':[3,5,7,10],
          'learning_rate': [0.05,0.1,0.15,0.2,0.25,0.3],
          'max_depth': [3,4,5,6,7,8,10,12,15], 
          'reg_alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
         }

In [ ]:
params = {'colsample_bytree': [0.1,0.2],
          #'subsample': [0.1,0.2,0.3,0.4],
          #'colsample_bylevel':[0.1,0.2,0.3,0.4],
          #'min_child_weight ':[1,3,5,7] ,
          'reg_lambda':[0.1,0.2,0.3]
          #'n_estimator':[3,5,],
          #'learning_rate': [0.05,0.1,0.15,0.2,0.25,0.3],
          #'max_depth': [3,4,5,6,7,8,10,12,15], 
          #'reg_alpha': [0.1,0.2,0.3]
         }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
scoring='r2'
#scoring='neg_mean_squared_error'
random_search=RandomizedSearchCV(xg_reg,param_distributions=params,scoring=scoring,n_iter=20,cv=5,verbose=3)

In [ ]:
random_search.fit(X_train,y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [50]:
best_gb=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.4,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=np.nan,
             monotone_constraints='()', n_estimator=5, n_estimators=100,
             n_jobs=0, num_parallel_tree=1, objective='reg:squarederror',
             random_state=0, reg_alpha=0.4, reg_lambda=0.1, scale_pos_weight=1,
             subsample=0.3, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [44]:
from sklearn.model_selection import cross_val_score as cvs
score=cvs(best_gb,X_train,y_train,cv=5,scoring='r2')

[17:10:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:13:06] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:15:59] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in langu

C:\Users\Raja Sekhar\Anaconda33\lib\site-packages\sklearn\model_selection\_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


MemoryError: 

In [45]:
score

array([0.49383491, 0.31226366, 0.29071351, 0.3636798 , 0.29428447])

In [46]:
score.mean()

0.3509552712445454

In [52]:
best_gb.fit(X_train,y_train)
preds = best_gb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

[17:42:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


RMSE: 0.623945


In [53]:
from sklearn.metrics import r2_score
r2_score(y_test,preds)

0.0802965281857646